<a href="https://colab.research.google.com/github/nish128/Customer-Segmentation/blob/main/rfm_feature_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Recency: Days since the customer last purchased

Frequency: How often they bought (unique invoices)

Monetary: Total money they spent



In [5]:
#import libraries
import pandas as pd
from datetime import datetime

# Step 1: Load data
df = pd.read_csv("E-Commerce data.csv", encoding='ISO-8859-1')

# Step 2: Data Cleaning
df = df[df['CustomerID'].notnull()]                # Remove missing CustomerID
df = df[df['Quantity'] > 0]                        # Remove negative quantities
df = df[df['UnitPrice'] > 0]                       # Remove zero/negative prices
# Convert date with specified format, coercing errors
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], format='%m/%d/%Y %H:%M', errors='coerce')
df.dropna(subset=['InvoiceDate'], inplace=True) # Remove rows where InvoiceDate is NaT
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']    # Create TotalPrice column

# Step 3: Define Reference Date (e.g., 1 day after last transaction)
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# Step 4: RFM Calculation (grouped by CustomerID)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',                                     # Frequency
    'TotalPrice': 'sum'                                         # Monetary
}).reset_index()

# Step 5: Rename Columns
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Step 6: View and Save
print(rfm.head())
rfm.to_csv("RFM_Table.csv", index=False)
print("✅ RFM table saved as 'RFM_Table.csv'")




   CustomerID  Recency  Frequency  Monetary
0     12346.0      269          1  77183.60
1     12347.0      260          1    475.39
2     12348.0       19          3   1430.24
3     12352.0       15          5   1209.66
4     12353.0      147          1     89.00
✅ RFM table saved as 'RFM_Table.csv'
Expected rows in RFM table: 3806
